In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
import joblib  # to load your trained SVM model

In [2]:

# ================================
# 1. Load Pre-trained VGG16 Model
# ================================
# Use the convolutional base only, exclude the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 8s 0us/step


In [3]:
# You can apply GlobalAveragePooling2D to match your deep feature size
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

In [4]:

model = Sequential([
    base_model,
    GlobalAveragePooling2D()  # Output: (None, 512)
])


In [5]:

# =====================================
# 2. Function to Extract Deep Features
# =====================================
def extract_deep_features(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    # Get deep features
    features = model.predict(img_data)

    # Flatten to 1D vector
    features_flat = features.flatten()

    # Reshape as a single sample for prediction
    return features_flat.reshape(1, -1)


In [ ]:

# =================================
# 3. Normalize + Predict with SVM
# =================================

# Load trained StandardScaler and SVM (assuming you saved them earlier)
scaler = joblib.load('scaler.pkl')        # StandardScaler() used during training
svm_model = joblib.load('svm_model.pkl')  # Your trained SVM

In [ ]:

def predict_real_or_fake(img_path):
    deep_feat = extract_deep_features(img_path)
    deep_feat_scaled = scaler.transform(deep_feat)
    prediction = svm_model.predict(deep_feat_scaled)
    return "Real" if prediction[0] == "real" else "Fake"

In [ ]:
result = predict_real_or_fake("path_to_your_custom_image.jpg")
print("Prediction:", result)